<b>Google Colab</b> <a href="https://colab.research.google.com/github/kirillzyusko/deeplearning/blob/master/7/lab.ipynb">link</a>

Authorize google + kaggle

In [1]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Download 100%.


Be sure, that we authorized and have an access to kaggle

In [2]:
%ls /content/.kaggle/

kaggle.json*


# **Part 1: Download dataset, extract**

Download dataset:

In [3]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle datasets download lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p /content/kaggle/imdb

 70% 18.0M/25.7M [00:00<00:00, 28.6MB/s]
100% 25.7M/25.7M [00:00<00:00, 40.5MB/s]


Extract .zip

In [0]:
!unzip kaggle/imdb/imdb-dataset-of-50k-movie-reviews.zip -d data

Archive:  kaggle/imdb/imdb-dataset-of-50k-movie-reviews.zip
  inflating: data/IMDB Dataset.csv   


Files:

In [2]:
%ls data

'IMDB Dataset.csv'


Read data using pandas:

In [5]:
import pandas as pd

imdb_ds = pd.read_csv('data/IMDB Dataset.csv')
print(imdb_ds.shape)

imdb_ds.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [6]:
!wget https://raw.githubusercontent.com/cdipaolo/sentiment/master/datasets/imdb.vocab

--2020-03-18 13:31:22--  https://raw.githubusercontent.com/cdipaolo/sentiment/master/datasets/imdb.vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 845980 (826K) [text/plain]
Saving to: ‘imdb.vocab’

imdb.vocab          100%[===================>] 826.15K  --.-KB/s    in 0.09s   

2020-03-18 13:31:22 (8.77 MB/s) - ‘imdb.vocab’ saved [845980/845980]



In [0]:
with open("imdb.vocab", "r") as file:
  data = file.read()
  print(data)

In [9]:
import tensorflow_datasets as tfds

dataset, info = tfds.load('imdb_reviews/subwords8k', with_info=True,
                          as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteO7H2P2/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteO7H2P2/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteO7H2P2/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [0]:
train_dataset, test_dataset = dataset['train'], dataset['test']

In [13]:
encoder = info.features['text'].encoder

print('Vocabulary size: {}'.format(encoder.vocab_size))

Vocabulary size: 8185


# **Part 2:**

# **Part 3:**

# **Part 4:**